In [709]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import GridSearchCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

План работы:
1. Изучить базовый код:
    В базовом коде имеем:
    1.1. Тренировочный и тестовый датасеты помечены с учетом того чтобы в последствии их разъеденить обратно. Затем они слиты в единый датасет чтобы подвергнуться предобработке.
    1.2. Обработаны бинарные переменные
    1.3. Обработаны категорийные переменные
    1.4. Обратное разделение на тренировочный и тестовый датасеты
    1.5. Разделение тренировочного датасета на обучающий и валидационный датасеты.
    1.6. Обучение модели
    1.7. Вывод метрик
2. Провести EDA:
3. Предобработка:
4. Добавить дополнительные признаки
5. Обучение модели 
6. Оценка модели
7. Рефлексия над проектом.

In [710]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [711]:
sample_submission.shape

In [712]:
df_test.shape

In [713]:
df_train.info()

In [714]:
df_train.head(5)

In [715]:
df_test.info()

In [716]:
sample_submission.head(5)

In [717]:
sample_submission.info()

In [718]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения default, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [719]:
data.nunique(dropna=False)

In [720]:
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
cat_cols = ['education', 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [721]:
print(data.default.value_counts())
data.score_bki.describe()

In [722]:
data['default'].value_counts(ascending=True).plot(kind='barh')

In [723]:
# Удаляем столбцы которые бесполезные для обучения модели
data.drop(['client_id','app_date',], axis = 1, inplace=True)

In [724]:
# посмотрим на распределение числовых признаков
for i in num_cols:
    plt.figure()
    sns.distplot(data[i][data[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [725]:
# Из графиков видим что первое это что наши числовые значения сильно несбалансированы
# по абсолютным значениям. Второе что имеются многочисленные выбросы. 
# Для исправления воспользуемся RobustScaler после того как разделим на обучающую выборку и
# валидационную. Сразу на всем датасете делать нельзя так как модель будет подстраиваться под тестовую часть

sns.set_theme(style="whitegrid")
ax = sns.boxplot(data=data[num_cols])

In [726]:
# Выводим корреляционную матрицу, видим что наши числовые признаки имееют слабую 
# линейную корреляцию между собой и проредить столбцы с высокой кореляцией нет вариантов.
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1)

In [727]:
# Обрабатываем категорийные признаки
data = pd.get_dummies(data, columns=['education'], dummy_na=True)

In [728]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for column in bin_cols:
    data[column] = le.fit_transform(data[column])
    
columns = ['first_time', 'sna', 'work_address', 'home_address', 'region_rating']

for column in columns:
    data[column] = le.fit_transform(data[column])

In [729]:
data.info()

In [730]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

# Уменьшаем количество доминирующего класса путем отрезания n-первых строк, где n количество примеров класса меньшинства
n = train_data.default.value_counts()[1]
vis_data_1 = train_data[(train_data.default == 1)]
vis_data_plus = train_data[(train_data.default == 0)].iloc[0:n]
df = pd.concat([vis_data_plus, vis_data_1], axis=0, ignore_index=True)

# Увеличиваем количество записей класса меньшинства путем n-раз добавления 
#n = train_data.default.value_counts()[1]
#m = train_data.default.value_counts()[0]
#l = m // n
#vis_data_1 = train_data[(train_data.default == 1)]
#vis_data_plus = train_data[(train_data.default == 0)]
#vis_data_2 = pd.concat([vis_data_1, vis_data_1], axis=0, ignore_index=True)
#for i in range(l-1):
#    vis_data_2 = pd.concat([vis_data_2, vis_data_1], axis=0, ignore_index=True)
#df = pd.concat([vis_data_plus, vis_data_2], axis=0, ignore_index=True)
    
y = df['default'].values  # наш таргет
X = df.drop(['default'], axis=1)

In [731]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
from sklearn.model_selection import train_test_split

# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [732]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [733]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(data=X_train)

In [734]:
# Делаем для уменьшения влияния выбросов
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [735]:
# делаем регуляризацию для уменьшения переобучения модели
from sklearn.preprocessing import Normalizer
normal = Normalizer(norm='l1')
X_train = normal.fit_transform(X_train)
X_test = normal.transform(X_test)

In [736]:
ax = sns.boxplot(data=X_train)

In [737]:
# Импортируем необходимые библиотеки:
from sklearn.linear_model import LogisticRegression # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [738]:
logreg = LogisticRegression(solver='liblinear', C = 0.1, max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [739]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test, y_pred)
print(classification_report)

Метрики полученные при выравниивании долей класса путем n-раз добавления класса меньшинства

                precision    recall  f1-score   support

           0       0.66      0.67      0.67     13022
           1       0.67      0.65      0.66     12985

    accuracy                           0.66     26007
   macro avg       0.66      0.66      0.66     26007
weighted avg       0.66      0.66      0.66     26007

Метрики полученые после выравнивания долей классов путем обрезки доминирующего

                   precision    recall  f1-score   support

           0       0.68      0.69      0.68      1887
           1       0.68      0.66      0.67      1862

    accuracy                           0.68      3749
   macro avg       0.68      0.68      0.68      3749
weighted avg       0.68      0.68      0.68      3749

Первоначально полученные метрики

precision    recall  f1-score   support

           0       0.88      1.00      0.93     12933
           1       0.50      0.00      0.00      1827

    accuracy                           0.88     14760
   macro avg       0.69      0.50      0.47     14760
weighted avg       0.83      0.88      0.82     14760

In [740]:
# Выводы: 
# 1. Лучшие результаты метрики получились при усечении доминирующего класса
# 2. RobustScaler и регуляризация улучшают метрики
# 3. Подбор гиперпараметров пробовал подбирать вручную, C = 0.1 дало улучшение метрик


In [741]:
# Делаем RobustScaler и нормализацию на всех данных
scaler_X = RobustScaler()
X = scaler_X.fit_transform(X)

normal_X = Normalizer(norm='l1')
X = normal_X.fit_transform(X)


In [743]:
# если качество нас устраивает, обучаем финальную модель на всех обучающи данных
logreg_final = LogisticRegression(solver='liblinear', C = 0.1, max_iter=1000)
logreg_final.fit(X, y)

In [744]:
predict_submission = logreg_final.predict(test_data)

In [745]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [ ]:
# Вот тут случилась непонятная вещь с которой я так и не смог разобраться. 
# При обучении модели на всех обучающих данных модель перестала нормально предсказывать
# произошло это после того как полных обучающих данных я применил RobustScaler и Normalizer
# Нужна помощь!

In [746]:
sample_submission.describe()

In [747]:
!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Ilya Galuzin"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'